<a href="https://colab.research.google.com/github/ajboyd2/salmon/blob/master/paper_outputs/timing_comparison_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip uninstall -y numpy
%pip install salmon-lm
%pip install statsmodels
%pip install rpy2
exit()  # Restart runtime after installing correct packages

Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 46.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.0 which is incompatible.
tensorflow 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.26.0 which is incompatible.


In [1]:
import pandas as pd
import os
from salmon import *
%matplotlib inline
import statsmodels.api as sm
import statsmodels.formula.api as smf
import time
from tabulate import tabulate
import numpy as np
import cProfile
%load_ext rpy2.ipython

In [2]:
if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/harris.csv"):
    import urllib.request
    urllib.request.urlretrieve("https://raw.githubusercontent.com/ajboyd2/salmon/master/data/harris.csv", "data/harris.csv")

In [3]:
data = pd.read_csv("data/harris.csv")

In [4]:
%%R
data_r <- read.csv("data/harris.csv")

In [5]:
nums = np.random.randint(0,50000,size=(10000,1000))
data2 = pd.DataFrame(nums)
y = data2.loc[:,0]
X = data2.iloc[:,1:]
cols = data2.columns

In [6]:
%R -i data2,cols colnames(data2) <- unlist(cols);
%R rdf <- data.frame(data2);
%R y = rdf$X0
%R X = data.matrix(rdf[,-1])

/usr/local/lib/python3.10/dist-packages/rpy2/robjects/pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


array([[ 7156, 14664, 39575, ...,  3893, 29915, 13648],
       [ 8192,  1073,  9878, ..., 45973, 46834, 49519],
       [ 1987, 39350, 49257, ...,  5274,  6229, 43150],
       ...,
       [33252, 12266, 11074, ...,  1919,   843, 10281],
       [ 5311, 15676, 29418, ...,  3678, 43054, 28561],
       [ 3573, 25022, 26080, ..., 47989, 37335, 48559]], dtype=int32)

In [7]:
## salmon fit data
start_time = time.time()
explanatory = Q("Bsal")
response = Q("Sal77")
salmon_model = LinearModel(explanatory, response)
salmon_model.fit(data)
end_time = time.time()
elapsed_time_fit_sal = (end_time - start_time) * 1000

## statsmodels fit data
start_time = time.time()
statsmodels_results = smf.ols("Sal77 ~ Bsal", data=data).fit()
statsmodels_results.summary()
end_time = time.time()
elapsed_time_fit_sm = (end_time - start_time) * 1000

## salmon predict
start_time = time.time()
salmon_model.predict(data)
end_time = time.time()
elapsed_time_pred_sal = (end_time - start_time) * 1000

## statsmodels predict
start_time = time.time()
statsmodels_results.predict(data)
end_time = time.time()
elapsed_time_pred_sm = (end_time - start_time) * 1000

## salmon fit data2
start_time = time.time()
explanatory2 = C("Sex") + Q("Age") + Q("Exper") + Q("Senior") + Q("Bsal")
response2 = Q("Sal77")
salmon_model2 = LinearModel(explanatory2, response2)
salmon_model2.fit(data)
end_time = time.time()
elapsed_time_fit_sal2 = (end_time - start_time) * 1000

## statsmodels fit data2
start_time = time.time()
statsmodels_results2 = smf.ols("Sal77 ~ Sex + Age + Exper + Senior + Bsal", data=data).fit()
statsmodels_results2.summary()
end_time = time.time()
elapsed_time_fit_sm2 = (end_time - start_time) * 1000

## salmon predict2
start_time = time.time()
salmon_model2.predict(data)
end_time = time.time()
elapsed_time_pred_sal2 = (end_time - start_time) * 1000

## statsmodels predict2
start_time = time.time()
statsmodels_results2.predict(data)
end_time = time.time()
elapsed_time_pred_sm2 = (end_time - start_time) * 1000

## salmon fit data3
start_time = time.time()
explanatory3 = C("Sex") + Q("Bsal") +  Q("Exper") + C("Sex") * Q("Bsal") + C("Sex") * Q("Exper")
response3 = Q("Sal77")
salmon_model3 = LinearModel(explanatory3, response3)
salmon_model3.fit(data)
end_time = time.time()
elapsed_time_fit_sal3 = (end_time - start_time) * 1000

## statsmodels fit data3
start_time = time.time()
statsmodels_results3 = smf.ols('Sal77 ~ Sex + Bsal + Exper + Sex*Bsal + Sex*Exper', data=data).fit()
statsmodels_results3.summary()
end_time = time.time()
elapsed_time_fit_sm3 = (end_time - start_time) * 1000

## salmon predict3
start_time = time.time()
salmon_model3.predict(data)
end_time = time.time()
elapsed_time_pred_sal3 = (end_time - start_time) * 1000

## statsmodels predict3
start_time = time.time()
statsmodels_results3.predict(data)
end_time = time.time()
elapsed_time_pred_sm3 = (end_time - start_time) * 1000

# salmon fit data4
y = data2.loc[:,0]
X = data2.iloc[:,1:]
start_time = time.time()
salmon_model4 = LinearModel(X, y)
salmon_model4.fit()
end_time = time.time()
elapsed_time_fit_sal4 = (end_time - start_time) * 1000

## statsmodels fit data4
start_time = time.time()
statsmodels_results4 = sm.OLS(y, X).fit()
statsmodels_results4.summary()
end_time = time.time()
elapsed_time_fit_sm4 = (end_time - start_time) * 1000

# salmon predict4
start_time = time.time()
salmon_model4.predict(data2)
end_time = time.time()
elapsed_time_pred_sal4 = (end_time - start_time) * 1000

## statsmodels predict4
start_time = time.time()
statsmodels_results4.predict(X)
end_time = time.time()
elapsed_time_pred_sm4 = (end_time - start_time) * 1000

In [8]:
%%R

## R fit data
start_time = Sys.time()
R_model = lm("Sal77 ~ Bsal", data=data_r)
summary(R_model)
end_time = Sys.time()
elapsed_time_fit_r = (end_time - start_time) * 1000

## R predict
start_time = Sys.time()
predict(R_model)
end_time = Sys.time()
elapsed_time_pred_r = (end_time - start_time) * 1000

# R fit data2
start_time = Sys.time()
R_model2 = lm("Sal77 ~ Sex + Age + Exper + Senior + Bsal", data=data_r)
summary(R_model2)
end_time = Sys.time()
elapsed_time_fit_r2 = (end_time - start_time) * 1000

## R predict2
start_time = Sys.time()
predict(R_model2)
end_time = Sys.time()
elapsed_time_pred_r2 = (end_time - start_time) * 1000

# R fit data3
start_time = Sys.time()
R_model3 = lm("Sal77 ~ Sex + Bsal + Exper + Sex*Bsal + Sex*Exper", data=data_r)
summary(R_model3)
end_time = Sys.time()
elapsed_time_fit_r3 = (end_time - start_time) * 1000

## R predict3
start_time = Sys.time()
predict(R_model3)
end_time = Sys.time()
elapsed_time_pred_r3 = (end_time - start_time) * 1000

# R fit data4
start_time = Sys.time()
R_model4 = lm("y ~ .", data=rdf)
summary(R_model)
end_time = Sys.time()
elapsed_time_fit_r4 = (end_time - start_time) * 1000

## R predict4
start_time = Sys.time()
predict(R_model4)
end_time = Sys.time()
elapsed_time_pred_r4 = (end_time - start_time) * 1000

In [9]:
elapsed_time_fit_r = %Rget elapsed_time_fit_r
elapsed_time_pred_r = %Rget elapsed_time_pred_r
elapsed_time_fit_r2 = %Rget elapsed_time_fit_r2
elapsed_time_pred_r2 = %Rget elapsed_time_pred_r2
elapsed_time_fit_r3 = %Rget elapsed_time_fit_r3
elapsed_time_pred_r3 = %Rget elapsed_time_pred_r3
elapsed_time_fit_r4 = %Rget elapsed_time_fit_r4
elapsed_time_pred_r4 = %Rget elapsed_time_pred_r4

table = [
["Fit",  elapsed_time_fit_sal, elapsed_time_fit_sm, elapsed_time_fit_r[0]],
["Predict", elapsed_time_pred_sal, elapsed_time_pred_sm, elapsed_time_pred_r[0]],
["Fit2",  elapsed_time_fit_sal2, elapsed_time_fit_sm2, elapsed_time_fit_r2[0]],
["Predict2", elapsed_time_pred_sal2, elapsed_time_pred_sm2, elapsed_time_pred_r2[0]],
["Fit3",  elapsed_time_fit_sal3, elapsed_time_fit_sm3, elapsed_time_fit_r3[0]],
["Predict3", elapsed_time_pred_sal3, elapsed_time_pred_sm3, elapsed_time_pred_r3[0]],
["Fit4",  elapsed_time_fit_sal4, elapsed_time_fit_sm4, elapsed_time_fit_r4[0]],
["Predict4", elapsed_time_pred_sal4, elapsed_time_pred_sm4, elapsed_time_pred_r4[0]],
]
print (tabulate(table, headers=["Test", "Salmon", "statsmodels", "R"]))

Test           Salmon    statsmodels           R
--------  -----------  -------------  ----------
Fit         27.9691         12.0811     21.0137
Predict      0.671387        3.07631     2.52223
Fit2         6.6483         16.5944      7.80082
Predict2     1.50704         5.87392     1.14059
Fit3         6.58822        14.0831      2.53296
Predict3     2.9943          3.98731     4.37617
Fit4      3063.94        11648.2      8506.45
Predict4   233.897          40.2787    655.47
